In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os
from datetime import datetime, timedelta
import numpy as np
import urllib.parse

FILE_DB = "data/agg_brand_model_condition_city_2024_W28.parquet"

In [2]:
import firebase_admin as fa
from firebase_admin import firestore

secret = fa.credentials.Certificate('service-account.json')
app = fa.initialize_app(secret)
db = firestore.client()

In [ ]:
import yaml

def yaml_to_dict(yaml_file):
  with open(yaml_file, 'r') as file:
    yaml_data = yaml.safe_load(file)
    return yaml_data

# Example usage:
yaml_file_path = "options.yaml"
python_dict = yaml_to_dict(yaml_file_path)
print(python_dict)

{'brands': ['acura', 'alfa_romeo', 'am_general', 'aston_martin', 'audi', 'austin-healey', 'avanti_motors', 'bmw', 'bentley', 'bugatti', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'datsun', 'dodge', 'eagle', 'fiat', 'ferrari', 'fisker', 'ford', 'gmc', 'genesis', 'geo', 'honda', 'hummer', 'hyundai', 'ineos', 'infiniti', 'isuzu', 'jaguar', 'jeep', 'karma', 'kia', 'lamborghini', 'land_rover', 'lexus', 'lincoln', 'lotus', 'lucid', 'mg', 'mini', 'maserati', 'mazda', 'mclaren', 'mercedes-benz', 'mercury', 'mitsubishi', 'nissan', 'oldsmobile', 'opel', 'plymouth', 'polestar', 'pontiac', 'porsche', 'ram', 'rivian', 'rolls-royce', 'saab', 'saturn', 'scion', 'studebaker', 'subaru', 'suzuki', 'tesla', 'toyota', 'triumph', 'vinfast', 'volkswagen', 'volvo', 'smart', 'detomaso', 'delorean', 'packard', 'sunbeam', 'lancia', 'ac', 'american_motors', 'citroen', 'kaiser', 'koenigsegg', 'maybach', 'panoz', 'desoto', 'international', 'jensen', 'willys', 'facel-vega', 'hupmobile', 'nash', 'hudson', 'edsel'

In [8]:
ref = db.collection('metrics').document('carsInformation').set(python_dict, merge=True)

In [3]:
pr = pd.read_parquet(FILE_DB)
pr.head(3)

,brand,model,itemCondition,city,cat,sales_sum,sales_count
0,AC,Shelby Cobra,Used,Carrollton,None,863100.0,9
1,Acura,CL,Used,Corinth,luxurypassenger_standard,35982.0,9
2,Acura,CL,Used,Fort Collins,luxurypassenger_standard,35424.0,9


In [4]:
pr['model'].to_csv('models.csv')

### axis = 0  -> column wise
### axis = 1 -> row wise

In [5]:
from typing import List, TypedDict

class Metrics(TypedDict):
    new_avg_price: int
    used_avg_price: int
    total_avg_price: int
    new_sales_count: int
    used_sales_count: int
    total_sales_count: int


def get_avg(pd: pd.DataFrame):
    pd["avg_price"] = np.where(
        pd["sales_count"] > 0, (pd["sales_sum"] // pd["sales_count"]), 0
    )


def print_metrics(pd: pd.DataFrame):
    print("Dataframe: ", pd.head(10), sep="\n")
    print("shape: ", pd.shape)


def get_date_from_week(year, week):
    first_day_of_year = datetime(year, 1, 1)
    days_to_week = (week - 1) * 7
    # Adjust to the first day of the first week (ISO standard)
    if first_day_of_year.weekday() <= 3:  # If Jan 1 is Thursday or earlier
        start_of_week = first_day_of_year - timedelta(days=first_day_of_year.weekday())
    else:
        start_of_week = first_day_of_year + timedelta(
            days=7 - first_day_of_year.weekday()
        )
    date_of_week = start_of_week + timedelta(days=days_to_week)
    return date_of_week


def group_by_avg(
    pd: pd.DataFrame, group_by_columns: List[str], sum_by_columns: List[str]
) -> pd.DataFrame:
    result = pd.groupby(group_by_columns)[sum_by_columns].sum()
    get_avg(result)
    # print_metrics(result)

    return result


def normalize(name: str) -> str:
    return "_".join([urllib.parse.quote(x, safe='').lower() for x in name.split(" ")])

In [6]:
def update_table(x: pd.Series, key_names: int, output: dict[str, Metrics]):
    """
    name -> ($model, $city, $condition)
    row -> ($sales_sum, $sales_count, $avg_price)
    """
    key: tuple = x.name
    record_key: str = ""
    sales_count, avg_price = 0, 0

    if key_names == 2:
        key_1, key_2, condition = (
            normalize(key[0]),
            normalize(key[1]),
            normalize(key[2]),
        )
        record_key = key_1 + "$" + key_2
    else:
        key_1, condition = (
            normalize(key[0]),
            normalize(key[1]),
        )
        record_key = key_1

    sales_count, avg_price = x.iloc[1], x.iloc[2]

    metrics: Metrics = {}
    try:
        if record_key in output:
            metrics: Metrics = output[record_key]
        else:
            metrics: Metrics = Metrics(
                new_avg_price=0,
                used_avg_price=0,
                total_avg_price=0,
                new_sales_count=0,
                used_sales_count=0,
                total_sales_count=0,
            )

        if condition == "new":
            metrics["new_avg_price"] = avg_price
            metrics["new_sales_count"] = sales_count
        else:
            metrics["used_avg_price"] = avg_price
            metrics["used_sales_count"] = sales_count

        total_avg_price = (
            metrics["used_avg_price"] * metrics["used_sales_count"]
            + metrics["new_avg_price"] * metrics["new_sales_count"]
        )
        
        total_sale_count = metrics["used_sales_count"] + metrics["new_sales_count"]
    except Exception as e:
        print('Error: ', e, key, metrics, x.iloc[1], x.iloc[2], sep=' ')
        return

    if total_sale_count == 0:
        total_avg_price = 0
    else:
        total_avg_price = total_avg_price // total_sale_count

    metrics["total_avg_price"] = total_avg_price
    metrics["total_sales_count"] = total_sale_count

    output[record_key] = metrics

In [7]:
city_pd = group_by_avg(pr, ["city", "itemCondition"], ["sales_sum", "sales_count"])

city_result: dict[str, Metrics] = {}
city_pd.apply(update_table, axis=1, key_names=1, output=city_result)
# city_pd

city        itemCondition
Abbeville   New              None
            Used             None
Aberdeen    New              None
            Used             None
Abilene     New              None
                             ... 
Zelienople  New              None
            Used             None
Zion        Used             None
Zumbrota    New              None
            Used             None
Length: 5063, dtype: object

In [8]:
len(city_result.keys())

2960

In [19]:
model_db = {}
brand_db = {}
city_db = {}

In [20]:
def update_record(
    dp: dict[str, dict[str, Metrics]], key: str, info: Metrics, date: str
):
    if key in dp:
        dp[key][date] = info
    else:
        dp[key] = {}
        dp[key][date] = info

def db_update(db_name: str, output: dict[str, Metrics], date: str) -> None:
    for key, info in output.items():
        try:
            if db_name == "model_db":
                update_record(model_db, key, info, date)
            elif db_name == "brand_db":
                update_record(brand_db, key, info, date)
            else:
                update_record(city_db, key, info, date)
            # collection_ref = db.collection(db_name).document(key)
            # collection_ref.set(record, merge=True)
        except Exception as e:
            print("Error: ", e, db_name, key, info)

def run(folder_path: str):
    year = 2024
    aggregration_columns = ["sales_sum", "sales_count"]
    output: dict[str, Metrics] = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".parquet"):
            # Extract the week number from the file name
            parts = file_name.split("_")
            week_str = parts[-1][1:].replace(".parquet", "")  # Remove file extension
            if week_str.isdigit():  # Ensure it's a valid number
                week = int(week_str)
                date = get_date_from_week(year, week).date().strftime("%Y-%m-%d")

                parquet_data = pd.read_parquet(os.path.join(folder_path, file_name))

                # brand_db:
                brand_pd = group_by_avg(
                    parquet_data,
                    ["brand", "itemCondition"],
                    aggregration_columns,
                )
                output = {}
                brand_pd.apply(update_table, axis=1, key_names=1, output=output)
                db_update(db_name="brand_db", output=output, date=date)

                # model_db:
                model_pd = group_by_avg(
                    parquet_data,
                    ["model", "itemCondition"],
                    aggregration_columns,
                )
                output = {}
                model_pd.apply(update_table, axis=1, key_names=1, output=output)
                db_update(db_name="model_db", output=output, date=date)

                # city_db:
                city_pd = group_by_avg(
                    parquet_data,
                    ["city", "itemCondition"],
                    aggregration_columns,
                )
                output = {}
                city_pd.apply(update_table, axis=1, key_names=1, output=output)
                db_update(db_name="city_db", output=output, date=date)


In [21]:
run('./data/')

In [22]:
import yaml

data = {
    'cities': list(city_db.keys()),
    'models': list(model_db.keys()),
    'brands': list(brand_db.keys())
}

with open('options.yaml', 'w') as fp:
    yaml.dump(data, fp)

In [ ]:
# batch = db.batch()
# for city, info in city_db.items():
#    ref = db.collection('cities_db').document(city)
#    batch.set(ref, info, merge=True)

# batch.commit()

# batch_size = 500
# for i in range(0, len(data_to_add), batch_size):
#     batch = db.batch()
#     for data in data_to_add[i:i + batch_size]:
#         doc_ref = collection_ref.document()  # Auto-generate document ID
#         batch.set(doc_ref, data)

#     batch.commit()

Line Graph:

1. Brands -> brands_avg table

```json
    {
        "brand1": {
            "10/12/2024": { 
                "new_price": 20000,
                "old_price": 10000,
                "total_price": 40000,
                "new_sales_count": "xx",
                "old_sales_count": "xx"
            }
        }
    }
```

2. Models -> models_avg table
```json
    {
        "model1": {
            "10/12/2024": {
                "new_price": 20000,
                "old_price": 10000,
                "total_price": 40000,
                "new_sales_count": "xx",
                "old_sales_count": "xx"
            }
        }
    }
```
3. brands & models -> models_avg table
```json
    {
        "brand_model_1": {
            "10/12/2024": {
                "new_price": 20000,
                "old_price": 10000,
                "total_price": 40000,
                "new_sales_count": "xx",
                "old_sales_count": "xx"
            }
        }
    }
```
4. brand & city -> brand_city_avg table
```json
    {
        "brand_city_1": {
            "10/12/2024": {
                "new_price": 20000,
                "old_price": 10000,
                "total_price": 40000,
                "new_sales_count": "xx",
                "old_sales_count": "xx"
            }
        }
    }
```
5. model & city -> model_city_avg table
```json
    {
        "model_city_1": {
            "10/12/2024": {
                "new_price": 20000,
                "old_price": 10000,
                "total_price": 40000,
                "new_sales_count": "xx",
                "old_sales_count": "xx"
            }
        }
    }
```


brand, model, city and condition:

1. count of sales | car_condition  - over brands, models, cities, brand_model, brand_city, model_city
2. 